In [ ]:
# -*- coding: utf-8 -*-
"""
SaFASiLe: Secure and Federated learning with American Sign Language
coded by Richárd Ádám Vécsey Dr. (Richard)
Udacity Secure and Private AI Final Keystone Challange


SUMMARY:
This program learns gestures from american sign language (ASL). The dataset comes from Kaggle. With this program you are
able to train a neural network to identify hand gestures from ASL. Use it as a module your code or a standalone program.
My goal was bulding a small and fast neural network. You are very welcome to rebuild it. See variables part to refine the
code.

This program contains 3 big parts:
    - neural network
    - federated learning
    - secure learning

The core concepts meet with the requirements from SPAIC Keystone Challange: ND185, Lesson 9, Chapter 8


REQUIREMENTS:
PySyft 0.1.16a1
PyTorch 1.0.1
TorchVision 0.2.2


INSTRUCTIONS:
You have to save the trained model if you want to use it next time.
For saving you have to use the 'torch.save(model.state_dict(), PATH)' process.
For loading you have to use the 'torch.save(model.state_dict(), PATH)' process.
PATH is the path of your model.

    
VARIABLES:
***************
batch size:    Batch size during training. It's better if the size is higher than the number of result categories.
epochs:        Count of training epoch Higher numbers aren't always better since overfitting. Watch out, this model could be
               overfitted over 35-45 epoch. If you want to prevent this, use higher dropout value, or make a bigger model. 
               Default value: 1
learning_rate: Learning rate of optimizer. I use ADAM. Default value: 0.0001
size:          One dimension from the size of input pictures (28x28). You have to change this if you use other source with
               different sizes. Be careful, you have to transform your data before load it. Default value: 28
num_workers:   Number of workres during training. Default value: 1
***************
hidden_1:      Number of hidden layers. Be careful, this model contains fully connected layers. Default value: 256
hidden_2:      Number of hidden layers. Be careful, this model contains fully connected layers. Default value: 256
output:        Number of result categories. If you want to predict more gestures, you have to change it a higher value.
               Default value: 25
dropout:       Value of dropout rate, where 0=0% (no dropout) and 1=100%. The value must be between 0 and 1. Higher value
               means higher dropping to prevent overfitting. However higher value causes longen the training process.
               Default value: 0.2
***************


LINKS:
dataset: https://www.kaggle.com/datamunge/sign-language-mnist
PyTorch documentation: https://pytorch.org/docs/stable/index.html
PySyft: https://github.com/OpenMined/PySyft


SPECIAL THANKS:
My best friend, Axel helps me a lot about this code. He supervised the coding process and made me motivated. He is a deep
learning coder and participant of another Udacity course names Deep Learning with PyyTorch Nanodegree.


OTHER:
@author: Richárd Ádám Vécsey Dr. (Richard)
@email: richard@hyperrixel.com
@github: https://github.com/richardvecsey

@contributed partner: Axel Ország-Krisz Dr. (Axel)
@email: axel@hyperrixel.com
@github: https://github.com/okaxel
"""


__author__ = 'Richárd Ádám Vécsey Dr.'

__version__ = '1.0'


import syft as sy

import torch

from torch import nn, optim

from torchvision import transforms, models


batch_size = 75
epochs = 1
learning_rate = 0.0001
size = 28
num_workers = 0


# Device sets on automaticly. If you want to use cpu only, use the next line:
# device = torch.device('cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




# Defining Neural Network architecture
class SLCatcher(nn.Module):

    def __init__(self):

        super(SLCatcher, self).__init__()

        hidden_1 = 256
        hidden_2 = 256
        output = 25
        dropout = 0.2

        self.fc1 = nn.Linear(size * size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, output)
        self.dropout = nn.Dropout(dropout)



    def forward(self, x):

        x = x.view(-1, size * size)
        x = torch.relu(self.dropout(self.fc1(x)))
        x = torch.relu(self.dropout(self.fc2(x)))
        x = self.fc3(x)

        return x


    
def loadmnist():

    target_test = list()
    target_train = list()
    data_test = list()
    data_train = list()

    # Loading MNIST data
    with open('./dataset/sign_mnist_test.csv', 'r') as _data_test_loaded:
        _data_test_temp = _data_test_loaded.readlines()

    with open('./dataset/sign_mnist_train.csv', 'r') as _data_train_loaded:
        _data_train_temp = _data_train_loaded.readlines()

    is_first = True

    for i in _data_test_temp:

        # Dropping first label-line
        if is_first:
            is_first = False
        else:
            i = i.split(',')
            _target_test = int(i[0])
            _data_test = [float(i[x]) / 255.0 for x in range(1, len(i))]

            target_test.append(_target_test)
            data_test.append(_data_test)

    is_first = True

    for i in _data_train_temp:

        # Dropping first label-line
        if is_first:
            is_first = False
        else:
            i = i.split(',')
            _target_train = int(i[0])
            _data_train = [float(i[x]) / 255.0 for x in range(1, len(i))]

            target_train.append(_target_train)
            data_train.append(_data_train)

    return data_test, target_test, data_train, target_train



def main():

    global batch_size
    global device
    global epochs
    global learning_rate
    global num_workers

    model = SLCatcher()

    # Waiting for Godot... to load data
    data_test, target_test, raw_data_train, raw_target_train = loadmnist()

    len_train = len(raw_data_train)
    len_test = len(data_test)

    # Autobots and Decepticons help to Transform source data to tensor
    data_test = torch.tensor(data_test)
    target_test = torch.tensor(target_test)
    raw_data_train = torch.tensor(raw_data_train)
    raw_target_train = torch.tensor(raw_target_train)
    
    data_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=num_workers)
    target_test = torch.utils.data.DataLoader(target_test, batch_size=batch_size, num_workers=num_workers)
    data_train = torch.utils.data.DataLoader(raw_data_train, batch_size=batch_size, num_workers=num_workers)
    target_train = torch.utils.data.DataLoader(raw_target_train, batch_size=batch_size, num_workers=num_workers)

    # Preparing train process
    model = model.to(device)
    model.train()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    
    # On the Road... Train
    for epoch in range(epochs):

        loss_train = float(0)
        true_counter = float(0)
        batch_counter = 0

        model.train()

        for data, target in zip(data_train, target_train):

            batch_counter += 1

            data = data.to(device)
            target = target.to(device)
            
            # Feel free to try enable modelzero_grad() on the next line:
            # model.zero_grad()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()*data.shape[0]
            _, indices = torch.max(output, 1)
            luck_counter = 0
            for _index, _target in zip(indices, target):
                if _index == _target:
                    true_counter += 1

        loss_train = loss_train / len_train
        accuracy_train = (true_counter * 100) / len_train

        print('Epoch {:2d} train     loss: {:.6f}'.format(epoch+1, loss_train))
        print('Epoch {:2d} train accuracy: {:.2f}'.format(epoch+1, accuracy_train))



        # Preparing test process
        model.eval()

        # Test
        loss_test = float(0)
        true_counter = float(0)

        for data, target in zip(data_test, target_test):
            data = data.to(device)
            target = target.to(device)

            #model.zero_grad()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss_test += loss.item()*data.shape[0]

            _, indices = torch.max(output, 1)
            for _index, _target in zip(indices, target):
                if _index == _target:
                    true_counter += 1

        loss_test = loss_test / len_test
        accuracy_test = (true_counter * 100) / len_test

        print('    Test loss: {:.6f}'.format(loss_test))
        print('Test accuracy: {:.2f}'.format(accuracy_test))
                
        # Here comes the Sun... and federated and secure learning too
        hook = sy.TorchHook(torch)
        vw1 = sy.VirtualWorker(hook, id="vw1").add_worker(sy.local_worker)
        vw2 = sy.VirtualWorker(hook, id="vw2").add_worker(sy.local_worker)
        secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)
        raw_data_train = raw_data_train.to(device)

        
        """
        This part of code necessary to secure learning. Running secure codes gives an empty AssertionError on the date
        17.08.2019. It comes from the installation process since pip install couldn't make installation process succesfully. 
        The current version of PySyft doesn't fit with current version of PyTorch,TorchVision and TensorFlow. If you want to
        use secure learning, you have to update these. I hope the next versions will be better. Be careful, "the name
        tf.Session is deprecated. Please use tf.compat.v1.Session instead"
        
        Secure Learning part begins:
        ---------------------------
        
        encrypted_model = model.fix_precision().share(vw1, vw2, crypto_provider=secure_worker)
        encrypted_data_train = raw_data_train.fix_precision().share(vw1, vw2, crypto_provider=secure_worker)
        # encrypted_target_train = raw_target_train.fix_precision().share(vw1, vw2, crypto_provider=secure_worker)
        encrypted_prediction = encrypted_model(encrypted_data_train)
        encrypted_prediction.get().float_precision()
        
        ---------------------------
        Secure Learning part ends
        """
        
        # Federated learning part: don't be confused with the variable names, it's without secure learning.
        encrypted_model = model.share(vw1, vw2, crypto_provider=secure_worker)
        encrypted_data_train = raw_data_train.share(vw1, vw2, crypto_provider=secure_worker)
        encrypted_prediction = encrypted_model(encrypted_data_train)
        encrypted_prediction.get()
        
        # That's all Folks!
        # I choose this printing method since this is pythonic :-)
        print('{}'.format('Finished.'))
        
        
# Main without "e" as this is a program not a state.
if __name__ == '__main__':
    main()